In [1]:
from pathlib import Path
from typing import List, Dict, Any, Optional


def parse_dkcpc_routes(path: str | Path) -> List[Dict[str, Any]]:
    """
    Parse DKCPC_2018-style route files.
    Returns list of routes with:
      { "name": ..., "color": ..., "points": [(lat_minutes, lon_minutes), ...] }
    """
    path = Path(path)
    routes: List[Dict[str, Any]] = []
    current_route: Optional[Dict[str, Any]] = None

    def _start_new_route():
        nonlocal current_route, routes
        if current_route is not None and current_route["points"]:
            routes.append(current_route)
        current_route = {
            "name": "uten navn",
            "color": None,
            "points": [],
            "metadata": {},
        }

    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for raw_line in f:
            line = raw_line.strip()
            if not line:
                continue

            if line.startswith("Ferdig forenklet"):
                _start_new_route()
                continue

            if line.startswith("Rute "):
                if current_route is None:
                    _start_new_route()
                current_route["name"] = line[len("Rute "):].strip()
                continue

            if line.startswith("Linjefarge "):
                if current_route is None:
                    _start_new_route()
                current_route["color"] = line[len("Linjefarge "):].strip()
                continue

            parts = line.split()
            if len(parts) >= 2:
                try:
                    x = float(parts[0])  # lat in minutes
                    y = float(parts[1])  # lon in minutes
                except ValueError:
                    continue

                if current_route is None:
                    _start_new_route()
                current_route["points"].append((x, y))

    if current_route is not None and current_route["points"]:
        routes.append(current_route)

    return routes


def export_bbox_points_to_txt(
    routes: List[Dict[str, Any]],
    bbox: list[float],
    out_path: str | Path,
) -> int:
    """
    Filter all points inside bbox, convert to degrees, and export to a TXT file.

    bbox format: [lat_max, lon_min, lat_min, lon_max]

    Output columns (comma-separated):
        lat_deg,lon_deg,route_name,color
    """
    out_path = Path(out_path)
    lat_max, lon_min, lat_min, lon_max = bbox

    rows = []

    for r in routes:
        name = r["name"]
        color = r["color"] if r["color"] is not None else ""

        for lat_min_val, lon_min_val in r["points"]:
            lat = lat_min_val / 60.0
            lon = lon_min_val / 60.0

            if not (lat_min <= lat <= lat_max and lon_min <= lon <= lon_max):
                continue

            rows.append((lat, lon, name, color))

    # Write to txt (CSV-style)
    with out_path.open("w", encoding="utf-8") as f:
        f.write("lat_deg,lon_deg,route_name,color\n")
        for lat, lon, name, color in rows:
            # escape commas in name if needed
            safe_name = name.replace(",", " ")
            safe_color = color.replace(",", " ")
            f.write(f"{lat:.8f},{lon:.8f},{safe_name},{safe_color}\n")

    return len(rows)


if __name__ == "__main__":
    txt_path = "DKCPC_2018.txt"  # input file
    routes = parse_dkcpc_routes(txt_path)
    print(f"Loaded {len(routes)} routes.")

    bbox = [57.58, 10.5, 57.12, 11.92]  # [lat_max, lon_min, lat_min, lon_max]

    n_points = export_bbox_points_to_txt(
        routes,
        bbox,
        out_path="dkcpc_points_bbox_deg.txt",
    )
    print(f"Exported {n_points} points to dkcpc_points_bbox_deg.txt")


Loaded 1 routes.
Exported 695 points to dkcpc_points_bbox_deg.txt
